In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need these
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_classification/imdb/main')

In [2]:
!pip install tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 66kB/s 
     |████████████████████████████████| 61kB 28.6MB/s 
     |████████████████████████████████| 419kB 50.5MB/s 
     |████████████████████████████████| 3.0MB 47.0MB/s 


In [3]:
import tensorflow as tf
import numpy as np
import pprint
import logging
import time

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-alpha0
GPU Enabled: True


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx


def get_idx(symbol2idx, symbol):
  return symbol2idx.get(symbol, len(symbol2idx))

In [0]:
def data_generator(f_paths, params):
  for f_path in f_paths:
    
    with open(f_path) as f:
      print('Reading', f_path)
      for line in f:
        line = line.rstrip()
        label, text = line.split('\t')
        text = text.split(' ')

        words = [get_idx(params['word2idx'], w) for w in text]
        if len(words) >= params['max_word_len']:
          words = words[:params['max_word_len']]
        else:
          words += [0] * (params['max_word_len'] - len(words))

        chars = []
        for w in text:
          temp = []
          for c in list(w):
            temp.append(get_idx(params['char2idx'], c))
          if len(temp) < params['max_char_len']:
            temp += [0] * (params['max_char_len'] - len(temp))
          else:
            temp = temp[:params['max_char_len']]
          chars.append(temp)
        if len(chars) >= params['max_word_len']:
          chars = chars[:params['max_word_len']]
        else:
          chars += [[0] * params['max_char_len']] * (params['max_word_len'] - len(chars))

        y = int(label)

        yield words, chars, y


def dataset(is_training, params):
  _shapes = ([params['max_word_len']], [params['max_word_len'], params['max_char_len']], ())
  _types = (tf.int32, tf.int32, tf.int32)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_paths'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.batch(params['batch_size'])
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_paths'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.batch(params['batch_size'])
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
class Model(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    
    self.char_embedding = tf.keras.layers.Embedding(len(params['char2idx'])+1, params['char_embed_size'])
    self.word_embedding = tf.Variable(np.load('../vocab/word.npy'),
                                      dtype=tf.float32,
                                      name='pretrained_glove',
                                      trainable=False,)
    
    self.embed_drop = tf.keras.layers.Dropout(params['dropout_rate'])
    self.embed_fc = tf.keras.layers.Dense(params['rnn_units'], tf.nn.elu)
    
    self.drop1 = tf.keras.layers.Dropout(params['dropout_rate'])
    self.drop2 = tf.keras.layers.Dropout(params['dropout_rate'])
    self.drop3 = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.char_cnn = tf.keras.layers.Conv1D(filters=params['rnn_units'], kernel_size=5, padding='same')
    self.word_rnn1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(params['rnn_units'], return_sequences=True))
    self.word_rnn2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(params['rnn_units'], return_sequences=True))
    self.word_rnn3 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(params['rnn_units'], return_sequences=True))
    
    self.fc_drop = tf.keras.layers.Dropout(params['dropout_rate'])
    self.fc = tf.keras.layers.Dense(params['rnn_units'], tf.nn.elu)
    
    self.out_linear = tf.keras.layers.Dense(2)
  
  
  def call(self, inputs, training=False):
    words, chars = inputs
    if words.dtype != tf.int32:
      words = tf.cast(words, tf.int32)
    
    batch_sz = tf.shape(words)[0]
    rnn_units = 2 * params['rnn_units']
    
    chars = self.char_embedding(chars)
    chars = tf.reshape(chars, (batch_sz*params['max_word_len'], params['max_char_len'], params['char_embed_size']))
    chars = self.char_cnn(chars)
    chars = tf.reduce_max(chars, 1)
    chars = tf.reshape(chars, (batch_sz, params['max_word_len'], params['rnn_units']))
    
    words = tf.nn.embedding_lookup(self.word_embedding, words)
    
    x = tf.concat((words, chars), axis=-1)
    x = self.embed_drop(x, training=training)
    x = self.embed_fc(x)
    
    x = tf.reshape(x, (batch_sz*10*10, 10, params['rnn_units']))
    x = self.drop1(x, training=training)
    x = self.word_rnn1(x)
    x = tf.reduce_max(x, 1)
    
    x = tf.reshape(x, (batch_sz*10, 10, rnn_units))
    x = self.drop2(x, training=training)
    x = self.word_rnn2(x)
    x = tf.reduce_max(x, 1)
    
    x = tf.reshape(x, (batch_sz, 10, rnn_units))
    x = self.drop3(x, training=training)
    x = self.word_rnn3(x)
    x = tf.reduce_max(x, 1)
    
    x = self.fc_drop(x, training=training)
    x = self.fc(x)
    
    x = self.out_linear(x)
    
    return x

In [0]:
params = {
  'train_paths': [
    '../data/train_bt_part1.txt',
    '../data/train_bt_part2.txt',
    '../data/train_bt_part3.txt',
    '../data/train_bt_part4.txt',
    '../data/train_bt_part5.txt',
    '../data/train_bt_part6.txt',
  ],
  'test_paths': [
    '../data/test.txt',
  ],
  'buffer_size': 25000*2,
  'num_labels': 2,
  'batch_size': 32,
  'max_word_len': 1000,
  'max_char_len': 10,
  'char_embed_size': 30,
  'rnn_units': 200,
  'dropout_rate': 0.2,
  'clip_norm': 10.,
  'num_patience': 3,
  'lr': 3e-4,
}

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True    

In [9]:
params['char2idx'] = get_vocab('../vocab/char.txt')
params['word2idx'] = get_vocab('../vocab/word.txt')

model = Model(params)
model.build(input_shape=[[None, params['max_word_len']], [None, params['max_word_len'], params['max_char_len']]])
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.95)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

history_acc = []
best_acc = .0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)


while True:
  # TRAINING
  for words, chars, labels in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model((words, chars), training=True)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
      loss = tf.reduce_mean(loss)
  
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    optim.apply_gradients(zip(grads, model.trainable_variables))

    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1

  # EVALUATION
  m = tf.keras.metrics.Accuracy()

  for words, chars, labels in dataset(is_training=False, params=params):
    logits = model((words, chars), training=False)
    y_pred = tf.argmax(logits, axis=-1)
    m.update_state(y_true=labels, y_pred=y_pred)
    
  acc = m.result().numpy()
  logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))
  history_acc.append(acc)
  
  if acc > best_acc:
    best_acc = acc
    # you can save model here
  logger.info("Best Accuracy: {:.3f}".format(best_acc))
  
  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

W0605 00:27:45.242788 140459296466816 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fbec0197908>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0605 00:27:45.249629 140459296466816 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fbec01c4240>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0605 00:27:45.255250 140459296466816 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fbec01c48d0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0605 00:27:45.261412 140459296466816 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fbec014e0f0>: Note that this layer is not optimized for performance. Please use tf.keras.laye

[('embedding/embeddings:0', TensorShape([80, 30])),
 ('dense/kernel:0', TensorShape([500, 200])),
 ('dense/bias:0', TensorShape([200])),
 ('conv1d/kernel:0', TensorShape([5, 30, 200])),
 ('conv1d/bias:0', TensorShape([200])),
 ('bidirectional/forward_unified_lstm/kernel:0', TensorShape([200, 800])),
 ('bidirectional/forward_unified_lstm/recurrent_kernel:0',
  TensorShape([200, 800])),
 ('bidirectional/forward_unified_lstm/bias:0', TensorShape([800])),
 ('bidirectional/backward_unified_lstm/kernel:0', TensorShape([200, 800])),
 ('bidirectional/backward_unified_lstm/recurrent_kernel:0',
  TensorShape([200, 800])),
 ('bidirectional/backward_unified_lstm/bias:0', TensorShape([800])),
 ('bidirectional_1/forward_unified_lstm_1/kernel:0', TensorShape([400, 800])),
 ('bidirectional_1/forward_unified_lstm_1/recurrent_kernel:0',
  TensorShape([200, 800])),
 ('bidirectional_1/forward_unified_lstm_1/bias:0', TensorShape([800])),
 ('bidirectional_1/backward_unified_lstm_1/kernel:0', TensorShape([40

I0605 00:29:23.460613 140459296466816 interactiveshell.py:2882] Step 0 | Loss: 0.6840 | Spent: 97.5 secs | LR: 0.000300
I0605 00:29:37.622488 140459296466816 interactiveshell.py:2882] Step 50 | Loss: 0.6701 | Spent: 14.2 secs | LR: 0.000299
I0605 00:29:51.841365 140459296466816 interactiveshell.py:2882] Step 100 | Loss: 0.4655 | Spent: 14.2 secs | LR: 0.000298
I0605 00:30:06.232839 140459296466816 interactiveshell.py:2882] Step 150 | Loss: 0.4130 | Spent: 14.4 secs | LR: 0.000298
I0605 00:30:20.708279 140459296466816 interactiveshell.py:2882] Step 200 | Loss: 0.4408 | Spent: 14.5 secs | LR: 0.000297
I0605 00:30:35.448922 140459296466816 interactiveshell.py:2882] Step 250 | Loss: 0.2938 | Spent: 14.7 secs | LR: 0.000296
I0605 00:30:49.981035 140459296466816 interactiveshell.py:2882] Step 300 | Loss: 0.4254 | Spent: 14.5 secs | LR: 0.000295
I0605 00:31:04.331852 140459296466816 interactiveshell.py:2882] Step 350 | Loss: 0.5776 | Spent: 14.3 secs | LR: 0.000295
I0605 00:31:18.665928 14045

Reading ../data/test.txt


I0605 00:39:47.533216 140459296466816 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.888
I0605 00:39:47.534427 140459296466816 interactiveshell.py:2882] Best Accuracy: 0.888


...

I0605 03:25:09.718789 140459296466816 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.920
I0605 03:25:09.720580 140459296466816 interactiveshell.py:2882] Best Accuracy: 0.921


Reading ../data/train_bt_part1.txt
Reading ../data/train_bt_part2.txt
Reading ../data/train_bt_part3.txt
Reading ../data/train_bt_part4.txt
Reading ../data/train_bt_part5.txt
Reading ../data/train_bt_part6.txt


I0605 03:26:37.661966 140459296466816 interactiveshell.py:2882] Step 23450 | Loss: 0.0098 | Spent: 274.8 secs | LR: 0.000090
I0605 03:26:52.330690 140459296466816 interactiveshell.py:2882] Step 23500 | Loss: 0.1614 | Spent: 14.7 secs | LR: 0.000090
I0605 03:27:06.965000 140459296466816 interactiveshell.py:2882] Step 23550 | Loss: 0.1570 | Spent: 14.6 secs | LR: 0.000090
I0605 03:27:21.430446 140459296466816 interactiveshell.py:2882] Step 23600 | Loss: 0.0425 | Spent: 14.5 secs | LR: 0.000089
I0605 03:27:35.723658 140459296466816 interactiveshell.py:2882] Step 23650 | Loss: 0.1117 | Spent: 14.3 secs | LR: 0.000089
I0605 03:27:50.021774 140459296466816 interactiveshell.py:2882] Step 23700 | Loss: 0.1158 | Spent: 14.3 secs | LR: 0.000089
I0605 03:28:04.338064 140459296466816 interactiveshell.py:2882] Step 23750 | Loss: 0.0758 | Spent: 14.3 secs | LR: 0.000089
I0605 03:28:18.670845 140459296466816 interactiveshell.py:2882] Step 23800 | Loss: 0.1716 | Spent: 14.3 secs | LR: 0.000088
I0605 0

Reading ../data/test.txt


I0605 03:36:56.952832 140459296466816 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.920
I0605 03:36:56.954186 140459296466816 interactiveshell.py:2882] Best Accuracy: 0.921
I0605 03:36:56.955011 140459296466816 interactiveshell.py:2882] Testing Accuracy not improved over 3 epochs, Early Stop
